In [27]:
import os
import yaml
import nbimporter
import warnings
import logging
import pandas as pd
from datetime import datetime, date
import pyspark
from pyspark.sql import SparkSession

from lab_database_manager import PgDBManager
from lab_schema_manager import SchemaManager
from lab_raw_yahoo import get_raw_yahoo, get_raw_yahoo_by_looping_groups
from lab_iceberg_manager import IcebergManager
from lab_iceberg_pg_operator import IcebergPgOperator


In [28]:
equity_type="etf"

# Get Finalytics Connetion and Create a PgDBManager

In [29]:
etl_start_time=datetime.now()
# Get finalytics connetion info
conn_config_file='cfg_connections.yaml'
pg_db="finalytics"
pg_db_mgr=PgDBManager(conn_config_file, pg_db)
pg_jdbc_url=pg_db_mgr.jdbc_url
pg_jdbc_properties =pg_db_mgr.jdbc_properties

### Get group_date_symbol_list from finalytics

In [30]:
# Get symbol_start_date_pairs from finalytics
group_size=100
query=f"SELECT group_id, group_start_date, symbol  FROM fin.ufn_etl_get_grouped_{equity_type}_eod_start_date({group_size}) where group_id <5 "
query_result=pg_db_mgr.get_sql_script_result_list(query)

In [31]:
print(len(query_result))

400


# Get Yahoo Hist Data

In [32]:
import_time = datetime.now()
yahoo_api="yahooquery"
hist_data=get_raw_yahoo_by_looping_groups(yahoo_api, query_result)
hist_data["import_time"] = pd.to_datetime(import_time).tz_localize(None)

Working on 1/4
An error occurred: HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/JKE?period1=1737676800&period2=1737936000&interval=1d&events=div%2Csplit&formatted=false&lang=en-US&region=US&corsDomain=finance.yahoo.com&crumb=wOvysY%2F41v. (Caused by ResponseError('too many 500 error responses'))
0 records in 1/4.
Sleeping for 2 seconds...
Working on 2/4
13514 records in 2/4.
Sleeping for 2 seconds...
Working on 3/4
8279 records in 3/4.
Sleeping for 2 seconds...
Working on 4/4
5684 records in 4/4.
Sleeping for 5 seconds...


lab_raw_yahoo.ipynb:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  "            hist_data = hist_data.reset_index()\n",


In [33]:
print(len(hist_data))

27477


# Create Spark Session

In [34]:
# Create Spark Session
conn_config_file="cfg_connections.yaml"
schema_config_file='cfg_schemas.yaml'
spark_app_name="raw_yfinance"
iceberg_raw_eod_table=f"nessie.raw.{equity_type}_eod_yahoo"
my_iceberg_manager=IcebergManager(conn_config_file, schema_config_file, spark_app_name) 
my_spark_session = my_iceberg_manager.get_spark_session()

### Create namespaces

In [35]:
my_spark_session.sql("CREATE NAMESPACE IF NOT EXISTS nessie.raw;")
my_spark_session.sql("CREATE NAMESPACE IF NOT EXISTS nessie.fin;")

DataFrame[]

### Load Iceberg Raw Table

In [36]:

# create spark data frame
hist_df = my_spark_session.createDataFrame(hist_data)
my_iceberg_manager.truncate_iceberg_table(iceberg_raw_eod_table)
my_iceberg_manager.insert_into_iceberg_table(hist_df, iceberg_raw_eod_table)


Iceberg table nessie.raw.etf_eod_yahoo truncated successfully.


nessie.raw.etf_eod_yahoo was loaded with 27477 records, totally 27477 records.


# Load data from Iceberg to Pg with an IcebergPgOperator

In [37]:
pg_table=f"stage.{equity_type}_eod_quote_yahoo"
pg_truncate_script=f"TRUNCATE TABLE {pg_table}"
pg_db_mgr.execute_sql_script(pg_truncate_script)

my_iceberg_pg_operator=IcebergPgOperator(my_spark_session, pg_jdbc_url, pg_jdbc_properties)
jdbc_mode="append"
my_iceberg_pg_operator.insert_iceberg_data_into_pg(iceberg_raw_eod_table, pg_table, jdbc_mode)


# Merge Pg stage into fin

In [38]:
pg_merge_script = "call fin.usp_load_{equity_type}_eod();"
pg_db_mgr.execute_sql_script(pg_merge_script)

An error occurred: syntax error at or near "{"
LINE 1: call fin.usp_load_{equity_type}_eod();
                          ^



In [39]:
etl_end_time=datetime.now()
delta_time = etl_end_time - etl_start_time
print(delta_time)


0:01:00.138074
